<a href="https://colab.research.google.com/github/biswajitmohanty/genai/blob/main/TravelChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install langchain groq gradio langchain_community


In [28]:
!pip install --upgrade httpx certifi


In [35]:
# Import necessary libraries
import json
import gradio as gr
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq

import os
import warnings
warnings.filterwarnings('ignore')
# Replace with your Groq API key
os.environ["GROQ_API_KEY"] = "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"

# Instantiate Groq’s chat model (this code assumes the integration for "llama-3.1-8b-instant" is set up)
chat_model = ChatGroq(model_name="llama-3.1-8b-instant")

# -------------------------------
# 1. Summarization Chain
# -------------------------------
# Create a prompt template to instruct the model to summarize a travel story in 2-3 sentences.
summarize_prompt = PromptTemplate(
    input_variables=["story"],
    template="Summarize in 2-3 sentences:\n{story}"
)
summarize_chain = LLMChain(llm=chat_model, prompt=summarize_prompt)

def summarize_story(story: str) -> str:
    """
    Summarizes the travel story.
    """
    summary = summarize_chain.run(story=story)
    return summary

# -------------------------------
# 2. Named Entity Recognition (NER) Chain
# -------------------------------
# Create a prompt template to instruct the model to extract named entities.
# The prompt requests a JSON output with keys: word, type, start, and end.
ner_prompt = PromptTemplate(
    input_variables=["story"],
    template=(
        "Extract named entities (Person, Organization, Location) from the following travel story. "
        "Return a JSON list where each element has the keys: 'word', 'type', 'start', and 'end'.\n\n"
        "Story:\n{story}\n"
    )
)
ner_chain = LLMChain(llm=chat_model, prompt=ner_prompt)

def extract_entities(story: str) -> dict:
    """
    Runs NER on the travel story and returns a structure suitable for Gradio's HighlightedText.
    Expects the model to return a JSON list with each entry having 'word', 'type', 'start', 'end'.
    """
    result = ner_chain.run(story=story)
    try:
        entities_list = json.loads(result)
    except Exception as e:
        entities_list = []

    # Build a list of annotations: each tuple is (start_index, end_index, label).
    annotations = []
    for entity in entities_list:
        if all(key in entity for key in ("start", "end", "type")):
            annotations.append((entity["start"], entity["end"], entity["type"]))

    return {"text": story, "entities": annotations}

# -------------------------------
# 3. Advanced Chatbot Function
# -------------------------------
def travel_chatbot(user_input: str, history: list, system_prompt: str, temperature: float):
    """
    Maintains conversation history with a system prompt.
    Uses the chatbot model to respond and returns updated history.
    The function returns three outputs:
      - Clears the user input
      - Updates the chatbot display (list of message tuples)
      - Stores the conversation state for future turns
    """
    if history is None:
        history = []

    # Build conversation messages from history.
    conversation = [{"role": "system", "content": system_prompt}]
    for entry in history:
        # Each conversation turn: first the user then the assistant.
        conversation.append({"role": "user", "content": entry[0]})
        conversation.append({"role": "assistant", "content": entry[1]})
    # Append the new user query.
    conversation.append({"role": "user", "content": user_input})

    try:
        # Call the model; using predict_messages (similar to ChatOpenAI API)
        response = chat_model.predict_messages(conversation, temperature=temperature)
        bot_response = response[0].content
    except Exception as e:
        bot_response = f"Error: {e}"

    # Update conversation history by appending the latest turn.
    history.append((user_input, bot_response))

    # Return:
    # - an empty string to clear the textbox,
    # - the updated chat (history) to display,
    # - the updated state (history).
    return "", history, history

# -------------------------------
# 4. Building the Gradio Interface
# -------------------------------
with gr.Blocks() as demo:
    gr.Markdown("## Travel Story Analyzer and Chatbot")

    # Create three tabs: one for summarization, one for NER, and one for the chatbot.
    with gr.Tabs():
        # Tab 1: Summarization
        with gr.TabItem("Summarize Story"):
            gr.Markdown("Enter a travel story and get its summary.")
            story_input = gr.Textbox(lines=6, label="Travel Story", placeholder="Enter your travel story here...")
            summary_output = gr.Textbox(lines=4, label="Summary")
            summarize_button = gr.Button("Summarize")
            summarize_button.click(fn=summarize_story, inputs=story_input, outputs=summary_output)

        # Tab 2: Named Entity Recognition
        with gr.TabItem("Named Entity Recognition"):
            gr.Markdown("Enter a travel story to extract named entities.")
            ner_story_input = gr.Textbox(lines=6, label="Travel Story", placeholder="Enter your travel story here...")
            ner_output = gr.HighlightedText(label="Named Entities")
            ner_button = gr.Button("Extract Entities")
            ner_button.click(fn=extract_entities, inputs=ner_story_input, outputs=ner_output)

        # Tab 3: Travel Chatbot
        with gr.TabItem("Travel Chatbot"):
            gr.Markdown("Chat with the travel story assistant.")
            # A textbox for a system prompt to control chatbot behavior.
            system_prompt_input = gr.Textbox(
                label="System Prompt",
                value="You are an expert travel guide who has travelled the entire world."
            )
            # Temperature slider to control response randomness.
            temperature_slider = gr.Slider(minimum=0.1, maximum=1.0, value=0.7, step=0.1, label="Temperature")
            # Chatbot display area
            chatbot_display = gr.Chatbot(label="Travel Chatbot")
            # User input area.
            user_message = gr.Textbox(label="Your Message", placeholder="Ask something about travel stories...")
            # Hidden state to store conversation history.
            state = gr.State([])

            send_button = gr.Button("Send")
            # When the send button is clicked, call travel_chatbot.
            # Outputs: clear the user textbox, update chatbot history, and update state.
            send_button.click(
                fn=travel_chatbot,
                inputs=[user_message, state, system_prompt_input, temperature_slider],
                outputs=[user_message, chatbot_display, state]
            )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://53666a6c3aaea95daa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
